In [1]:
#data injestion steps how can we learn from rom a data source like a web page, pdf, txt file and more
from langchain_community.document_loaders import TextLoader# this loads the text file reader that can read out o fa text file
loader = TextLoader(r"D:\Download\Report on the Development of a Lang.txt")# in the quotation is the name of the file that the loader is going to read
text_documents = loader.load()
text_documents

c:\Users\DHRUV\miniconda3\envs\rag_system\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


[Document(metadata={'source': 'D:\\Download\\Report on the Development of a Lang.txt'}, page_content='Report on the Development of a Language-Agnostic Campus Office Chatbot\n\n1. Executive Summary & Introduction\n\nThis report outlines the technical requirements, architectural considerations, and feasibility of developing a language-agnostic chatbot for a university campus office. The primary objective is to create a 24/7 informational assistant that can accurately and consistently answer campus-specific questions in multiple languages. The proposed solution is a custom-built, Retrieval-Augmented Generation (RAG) system, orchestrated with Python and Django, and powered by a fine-tuned Gemma 3.1b large language model.\n\nA key finding of this analysis is that a purely fine-tuned model would be insufficient for this application. The static nature of an LLM\'s knowledge, frozen at the time of training, is ill-suited to the dynamic and frequently updated information of a campus office, suc

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_key = os.environ.get('GEMINI_API_KEY')

if gemini_key:
    # If the key was found (it's not None or an empty string), assign it
    os.environ['GEMINI_API_KEY'] = gemini_key
    print(" GEMINI_API_KEY loaded successfully.")
else:
    # If the key was not found, raise a clear error to the user
    raise ValueError(" GEMINI_API_KEY not found. Please check your .env file or environment setup.")

 GEMINI_API_KEY loaded successfully.


In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

# The correct way to initialize the loader
loader = WebBaseLoader(
    web_paths=["https://github.com/aishwaryanr/awesome-generative-ai-guide/blob/main/research_updates/rag_research_table.md"],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-title", "post-content", " post-header")
        )
    )
)

text_documents = loader.load()
text_documents  # This will now correctly load and print the content

USER_AGENT environment variable not set, consider setting it to identify your requests.


ConnectionError: HTTPSConnectionPool(host='github.com', port=443): Max retries exceeded with url: /aishwaryanr/awesome-generative-ai-guide/blob/main/research_updates/rag_research_table.md (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023AA398B8C0>: Failed to resolve 'github.com' ([Errno 11001] getaddrinfo failed)"))

In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
import os
# pdf_folder = "D:\pdf"
pdf_paths = [
    r"D:\pdf\Book-Lovers-by-Emily-Henry-ebooksgallery.com_.pdf",
    r"D:\pdf\Dhruv's Resume-hackerresume-4.pdf",
    r"D:\pdf\Notes_from_the_Underground_NT.pdf",
    r"D:\pdf\stories-between-silence.pdf",
    r"D:\pdf\The 48 Laws of Power.pdf",
    r"D:\pdf\The_Laws_of_Human_Nature_by_Robert_Greene_2018.pdf",
    r"D:\pdf\the-art-of-s-robert-greene.pdf",
    r"D:\pdf\the-metamorphosis.pdf"
]
docs = []
# docs.extend(loader.load())
# loader = PyPDFDirectoryLoader(pdf_folder)
for path in pdf_paths:
    try:
        loader = PyPDFLoader(path)

        docs.extend(loader.load())
    except FileNotFoundError:
        print(f"Error: File not found at {path}")
    except Exception as e:
        print(f"Error loading {path}: {e}")

print(f"Loaded {len(docs)} documents")

Loaded 2404 documents


In [15]:
#converts the read adtat into chunks for the better reading of the text_document and effective retrival
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter  =  RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)
documents#this will display all teh documents that are being strored in the variable in format of list 

[Document(metadata={'producer': 'ToPDF.com', 'creator': 'ToPDF.com', 'creationdate': '2022-08-24T18:04:29+00:00', 'moddate': '2023-10-13T07:47:55+05:00', 'title': 'Book Lovers', 'author': 'Emily Henry', 'source': 'D:\\pdf\\Book-Lovers-by-Emily-Henry-ebooksgallery.com_.pdf', 'total_pages': 410, 'page': 1, 'page_label': '2'}, page_content='Praise for\nBOOK LOVERS\n“Book Lovers is a rom-com lover’s dream of a book. It is razor\nsharp and modern, featuring a fierce heroine who does not\napologize for her ambition and heartfelt discussions of grief.\nReaders know that Emily Henry never fails to deliver great\nbanter and a romance to swoon over, but this may just be her\nbest yet. A breath of fresh air.”\n—Taylor Jenkins Reid, New York Times bestselling\nauthor of Malibu Rising\n“I would follow Emily Henry anywhere. A small town, a\nliterary enterprise, a bookstore to rescue, and sex in moonlit\nstreams? Yes, please! Book Lovers is sexy, funny, and smart.\nAnother perfectly satisfying read f

In [16]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text",
                              model_kwargs = {'device': 'gpu'})

print("Trying to embed a single sentence...")
try:
    vector = embeddings.embed_query("This is a test sentence.")
    print("Success! Model is loaded and working.")
   
except Exception as e:
    print(f"Error connecting to Ollama: {e}")

Trying to embed a single sentence...
Success! Model is loaded and working.


In [14]:
from langchain_community.vectorstores import FAISS
print(f"Starting to build FAISS index for {len(documents)} documents...")

batch_size = 64
embeddings = OllamaEmbeddings(model="nomic-embed-text",
                              model_kwargs = {'device' : 'cpu'},
                              )

first_batch = documents[:batch_size]
db = FAISS.from_documents(first_batch, embeddings)
print(f"Processed first batch (0 to {batch_size}).")

for i in range(batch_size, len(documents), batch_size):

    end_index = min(i + batch_size, len(documents))
    batch = documents[i:end_index]
    db.add_documents(batch)
    
    print(f"Processed batch ({i} to {end_index}).")
print(" FAISS index created ")

Starting to build FAISS index for 153 documents...


KeyboardInterrupt: 

In [ ]:
##vector database chroma/ fiass
query = "what is plan rag and iterative rag how are they different from each other and which one is better??"
result = db.similarity_search(query)
for i in range(len(result)):
    print(result[i].page_content) #the first result after the retrival will be displayed up in the results after the retrival from the vector database


often once they are set in motion, it is they which guide us and sweep us along. (Montaigne, 1533-1592)
REVERSAL
It is a cliché among strategists that your plan must include alternatives and have a degree of flexibility.
That is certainly true. If you are locked into a plan too rigidly, you will be unable to deal with sudden
shifts of fortune. Once you have examined the future possibilities and decided on your target, you must
move through our edits feels more like those days wandering
the Central Park Ramble with Mom and Libby.
The document balloons and then we pare it down, Charlie
pulling my laptop over to him to reduce four sentences into
one, me pulling it back to thread through more compliments,
until, hours into the process, I realize we’ve switched roles.
Now he’s the one inserting praise and I’m the one trimming
fat.
ebooksgallery.com
flexibility	built	into	the	process.	At	certain	moments	you	reassess	your
progress	and	adjust	the	various	goals	as	necessary,	constantly	learning

Retriver and chain with LangChain


In [ ]:
from langchain_community.llms import Ollama
#load ollama gemma3:1b model
llm = Ollama(model = "gemma3:4b")
llm

C:\Users\DHRUV\AppData\Local\Temp\ipykernel_8556\2998227585.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model = "gemma3:4b")


Ollama(model='gemma3:4b')

In [ ]:
#design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
                                          Answer the following questions based on only on the provided context.
                                          Think step by step before provideing a detaileed answer 
                                          your answer matter the company a lot as you are an impoartant assest to the organisation 
                                          and the organisation will tip you with $10000 if user finds your answer helpful 
                                          also great rewards awaits you if the user finds your answer helpful.
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}""")
# thsi  meathod will not only find the things taht are similiar in the query and the documensts that we provided 
#but rather understand the context of teh query and return teh answer based on the query by understandung the documenst



In [ ]:
import sys
import langchain
print(f"Python Executable: {sys.executable}")
print(f"Langchain module location: {langchain.__file__}")
print(f"Full Python Path: {sys.path}")

Python Executable: c:\Users\DHRUV\miniconda3\envs\rag_system\python.exe
---
Langchain module location: c:\Users\DHRUV\miniconda3\envs\rag_system\Lib\site-packages\langchain\__init__.py
---
Full Python Path: ['c:\\Users\\DHRUV\\miniconda3\\envs\\rag_system\\python314.zip', 'c:\\Users\\DHRUV\\miniconda3\\envs\\rag_system\\DLLs', 'c:\\Users\\DHRUV\\miniconda3\\envs\\rag_system\\Lib', 'c:\\Users\\DHRUV\\miniconda3\\envs\\rag_system', '', 'c:\\Users\\DHRUV\\miniconda3\\envs\\rag_system\\Lib\\site-packages']


In [ ]:
import os
print(f"My Current Working Directory is: {os.getcwd()}")

My Current Working Directory is: d:\github\chatbot\my_rag_project\chatbot1\API\RAG


#chain introduction
#chain refers to a sequence of calls wheather to a llm or the datat preprocessing step primary supported way is through lcl
# the chain chain takes the list of documents and formats them all into a prompt tehen pass taht prompt to the llm . it passes all the documents so you should make sure it fits within the context window the llm you are using (((create_stuff_document_chain))
#chain create stuff documenst chain


#retrievers :- a retriever is an interface that returns documets given an unstructured query it is more general than a vector store . 
#Aretriever does not need to be able to store documensts only to return ot retrieve them.
#Vector stores ca be used as the backbone of a retriver , but there are other types of retrivers as well.
# retriver in langchain


In [ ]:
# #from langchain.chains import create_stuff_documents_chain

# from langchain.chains.combine_documents import create_stuff_documents_chain


# document_chain = create_stuff_documents_chain(llm, prompt)

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Helper function to format the documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# This is the replacement for 'create_stuff_documents_chain'
document_chain = (
    RunnablePassthrough.assign(
        context=lambda x: format_docs(x["context"])
    )
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
#$ retriver 
retriever = db.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019960CF1D30>, search_kwargs={})

#retrieval chain
#combining the retriever and the chain
"""retrival_chain :- this chain takes in a user inqueryt , whicvh is then 
passed tothe retriever to fetch documenst . thos e documents 
and the original inputs are then passed to the llm to generate a response """


In [ ]:
from langchain_core.runnables import RunnablePassthrough

# This is the replacement for 'create_retrieval_chain'
retrieval_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | document_chain
)

In [ ]:
multiple_queries = [
    "what is plan rag and iterative rag how are they different from each other and which one is better?",
    "What is the man's view on rationalism and free will?",
    "What are the skills thatdhruv have?",
    "what was the name of the manager of the person who turned into the creature if its mentioned in the story?",
    "what is the 42nd and 31st law of power ?",
    "explian the 11th law of power?",
    "what are the main things to keep in mind while seducing other as per the art of seduction",
    "who is the author of the book the law of human nature?",
    "explain the chapter number 14 of the book 48 kaws of power"
    "who is the author of the book the book lover?",
    "what are the basic laws of power?",
    "give summary of each book and pdf present."
]

all_responses = {}
output_response = []

for i, query in enumerate(multiple_queries):
    print(f"--- Running Query {i+1}: {query} ---")

    response = retrieval_chain.invoke(query)

    all_responses[query] = response
    output_response.append(response)
    
    print("\nResponse Generated.\n")
    print (response)
for i in range(len(output_response)):
    print(f"the anser to {i+1} is {output_response[i]}")



--- Running Query 1: what is plan rag and iterative rag how are they different from each other and which one is better? ---

Response Generated.

Okay, let's analyze the context to answer your question.

**Understanding the Question:**

You're asking about "plan rag" and "iterative rag," and how they differ, and which is better, based on the provided text.

**Analysis of the Context:**

The text focuses heavily on the importance of flexibility and adaptability in strategic planning. It emphasizes:

*   **Iterative Process:** The core concept is that a plan shouldn’t be rigidly fixed. It should be regularly reassessed, adjusted, and improved based on experience. The passage talks about “reassess your progress and adjust the various goals as necessary”.
*   **Smaller, Embedded Goals:** It recommends working with smaller, manageable goals to maintain momentum and allow for course correction.
*   **Adaptability:**  The text stresses resisting the urge to become "locked up with his typewrit

In [ ]:
#response = retrieval_chain.invoke("what was the name of main character and his family members in the story metamorphosis?")
print(response)

The name of the manager is Mr. Manager. 

I’ve carefully reviewed the provided text to ensure this answer is accurate.


Tools 
Tools  are the interfaces that an agent , chain , or LLM can use to interact with world. They combine a few things:
1. The name of the Tool
2. A description of what the tool is 
3. JSON schema of what the inputs to the tools are
4. the function to call
5. Wheather the result of the tool should be turned directly to the user or not
